In [3]:
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

There are 279 patients having there own unique id

In [7]:
base_path = "dataset/IDC_regular_ps50_idx5/"
folder = os.listdir(base_path)
print(len(folder))

279


In [9]:
from glob import glob
imagePatches = glob('dataset/IDC_regular_ps50_idx5/**/*.png', recursive=True)
print(len(imagePatches)) # total no. of images

277524


In [17]:
for i in range(10):
    print(imagePatches[i])

dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1001_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1051_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1101_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1151_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1201_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1251_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1301_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1351_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1501_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1551_class0.png


In [20]:
imagePatches[277520]

'dataset/IDC_regular_ps50_idx5\\9383\\1\\9383_idx5_x2051_y951_class1.png'

In [21]:
import fnmatch
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

y = []
for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)
        

In [25]:
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y
images_df.head()

,images,labels
0,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
1,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
2,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
3,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
4,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0


In [26]:
images_df.to_csv('dataset/dataset.csv', header=False, index=False) 

In [29]:
images_df.shape

(277524, 2)

In [30]:
images_df.groupby("labels")["labels"].count()

labels
0    198738
1     78786
Name: labels, dtype: int64

In [32]:
# spliting the dataset into train test and validation parts
train, test = train_test_split(images_df, stratify=images_df.labels, test_size=0.33)
print(len(train),len(test))

185941 91583


In [35]:
train

,images,labels
76260,dataset/IDC_regular_ps50_idx5\12870\0\12870_id...,0
268291,dataset/IDC_regular_ps50_idx5\9323\1\9323_idx5...,1
147137,dataset/IDC_regular_ps50_idx5\14154\0\14154_id...,0
21418,dataset/IDC_regular_ps50_idx5\10278\0\10278_id...,0
38233,dataset/IDC_regular_ps50_idx5\10299\1\10299_id...,1
...,...,...
115731,dataset/IDC_regular_ps50_idx5\13018\1\13018_id...,1
212529,dataset/IDC_regular_ps50_idx5\8975\0\8975_idx5...,0
91492,dataset/IDC_regular_ps50_idx5\12896\0\12896_id...,0
26485,dataset/IDC_regular_ps50_idx5\10285\0\10285_id...,0


In [38]:
train, val = train_test_split(train , stratify=train.labels, test_size=0.2)
print(len(train),len(val))

148752 37189


In [41]:
# saving the data in .csv format in different directories
train.to_csv('dataset/train/train.csv', header=False, index=False)
test.to_csv('dataset/test/test.csv', header=False, index=False)
val.to_csv('dataset/validation/validation.csv', header=False, index=False)